##**2. Word2Vec**
1. 주어진 단어들을 word2vec 모델에 들어갈 수 있는 형태로 만듭니다.
2. CBOW, Skip-gram 모델을 각각 구현합니다.
3. 모델을 실제로 학습해보고 결과를 확인합니다.

### **필요 패키지 import**

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 53.4MB/s 
     |████████████████████████████████| 460kB 31.9MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from tqdm import tqdm
from konlpy.tag import Okt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

import torch
import copy
import numpy as np

### **데이터 전처리**



데이터를 확인하고 Word2Vec 형식에 맞게 전처리합니다.  
학습 데이터는 1번 실습과 동일하고, 테스트를 위한 단어를 아래와 같이 가정해봅시다.

In [3]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."       
]

test_words = ["음식", "맛", "서비스", "위생", "가격"] # -> 학습된 Word2Vec 모델로 test word의 벡터를 뽑아내는 것

Tokenization과 vocab을 만드는 과정은 이전 실습과 유사합니다.

In [4]:
tokenizer = Okt()

In [5]:
# 토크나이징
def make_tokenized(data):
  tokenized = []
  for sent in tqdm(data):
    tokens = tokenizer.morphs(sent, stem=True)
    tokenized.append(tokens)

  return tokenized

In [6]:
train_tokenized = make_tokenized(train_data)

100%|██████████| 10/10 [00:06<00:00,  1.59it/s]


In [14]:
train_tokenized[:4]

[['정말', '맛있다', '.', '추천', '하다', '.'],
 ['기대하다', '것', '보단', '별로', '이다', '.'],
 ['다',
  '좋다',
  '가격',
  '이',
  '너무',
  '비싸다',
  '다시',
  '가다',
  '싶다',
  '생각',
  '이',
  '안',
  '드네',
  '요',
  '.'],
 ['완전', '최고', '이다', '!', '재', '방문', '의사', '있다', '.']]

In [7]:
word_count = defaultdict(int)

for tokens in tqdm(train_tokenized):
  for token in tokens:
    word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 3840.59it/s]


In [8]:
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print(list(word_count))

[('.', 14), ('도', 7), ('이다', 4), ('좋다', 4), ('별로', 3), ('다', 3), ('이', 3), ('너무', 3), ('음식', 3), ('서비스', 3), ('하다', 2), ('방문', 2), ('위생', 2), ('좀', 2), ('더', 2), ('에', 2), ('조금', 2), ('정말', 1), ('맛있다', 1), ('추천', 1), ('기대하다', 1), ('것', 1), ('보단', 1), ('가격', 1), ('비싸다', 1), ('다시', 1), ('가다', 1), ('싶다', 1), ('생각', 1), ('안', 1), ('드네', 1), ('요', 1), ('완전', 1), ('최고', 1), ('!', 1), ('재', 1), ('의사', 1), ('있다', 1), ('만족스럽다', 1), ('상태', 1), ('가', 1), ('개선', 1), ('되다', 1), ('기르다', 1), ('바라다', 1), ('맛', 1), ('직원', 1), ('분들', 1), ('친절하다', 1), ('기념일', 1), ('분위기', 1), ('전반', 1), ('적', 1), ('으로', 1), ('짜다', 1), ('저', 1), ('는', 1), ('신경', 1), ('써다', 1), ('불쾌하다', 1)]


In [9]:
w2i = {}
for pair in tqdm(word_count):
  if pair[0] not in w2i:
    w2i[pair[0]] = len(w2i)

100%|██████████| 60/60 [00:00<00:00, 109607.25it/s]


In [10]:
print(train_tokenized)
print(w2i)

[['정말', '맛있다', '.', '추천', '하다', '.'], ['기대하다', '것', '보단', '별로', '이다', '.'], ['다', '좋다', '가격', '이', '너무', '비싸다', '다시', '가다', '싶다', '생각', '이', '안', '드네', '요', '.'], ['완전', '최고', '이다', '!', '재', '방문', '의사', '있다', '.'], ['음식', '도', '서비스', '도', '다', '만족스럽다', '.'], ['위생', '상태', '가', '좀', '별로', '이다', '.', '좀', '더', '개선', '되다', '기르다', '바라다', '.'], ['맛', '도', '좋다', '직원', '분들', '서비스', '도', '너무', '친절하다', '.'], ['기념일', '에', '방문', '하다', '음식', '도', '분위기', '도', '서비스', '도', '다', '좋다', '.'], ['전반', '적', '으로', '음식', '이', '너무', '짜다', '.', '저', '는', '별로', '이다', '.'], ['위생', '에', '조금', '더', '신경', '써다', '좋다', '.', '조금', '불쾌하다', '.']]
{'.': 0, '도': 1, '이다': 2, '좋다': 3, '별로': 4, '다': 5, '이': 6, '너무': 7, '음식': 8, '서비스': 9, '하다': 10, '방문': 11, '위생': 12, '좀': 13, '더': 14, '에': 15, '조금': 16, '정말': 17, '맛있다': 18, '추천': 19, '기대하다': 20, '것': 21, '보단': 22, '가격': 23, '비싸다': 24, '다시': 25, '가다': 26, '싶다': 27, '생각': 28, '안': 29, '드네': 30, '요': 31, '완전': 32, '최고': 33, '!': 34, '재': 35, '의사': 36, '있다': 37, '만족스럽다': 38, '상태

실제 모델에 들어가기 위한 input을 만들기 위해 `Dataset` 클래스를 정의합니다.

In [11]:
class CBOWDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.x.append(token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.y.append(id)

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수, 2 * window_size)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [12]:
class SkipGramDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.y += (token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.x += [id] * 2 * window_size

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

각 모델에 맞는 `Dataset` 객체를 생성합니다.

In [19]:
cbow_set = CBOWDataset(train_tokenized)
skipgram_set = SkipGramDataset(train_tokenized)
print(list(skipgram_set))

100%|██████████| 10/10 [00:00<00:00, 1241.43it/s]

[(tensor(0), tensor(17)), (tensor(0), tensor(18)), (tensor(0), tensor(19)), (tensor(0), tensor(10)), (tensor(19), tensor(18)), (tensor(19), tensor(0)), (tensor(19), tensor(10)), (tensor(19), tensor(0)), (tensor(22), tensor(20)), (tensor(22), tensor(21)), (tensor(22), tensor(4)), (tensor(22), tensor(2)), (tensor(4), tensor(21)), (tensor(4), tensor(22)), (tensor(4), tensor(2)), (tensor(4), tensor(0)), (tensor(23), tensor(5)), (tensor(23), tensor(3)), (tensor(23), tensor(6)), (tensor(23), tensor(7)), (tensor(6), tensor(3)), (tensor(6), tensor(23)), (tensor(6), tensor(7)), (tensor(6), tensor(24)), (tensor(7), tensor(23)), (tensor(7), tensor(6)), (tensor(7), tensor(24)), (tensor(7), tensor(25)), (tensor(24), tensor(6)), (tensor(24), tensor(7)), (tensor(24), tensor(25)), (tensor(24), tensor(26)), (tensor(25), tensor(7)), (tensor(25), tensor(24)), (tensor(25), tensor(26)), (tensor(25), tensor(27)), (tensor(26), tensor(24)), (tensor(26), tensor(25)), (tensor(26), tensor(27)), (tensor(26), tens

### **모델 Class 구현**

차례대로 두 가지 Word2Vec 모델을 구현합니다.  


*   `self.embedding`: `vocab_size` 크기의 one-hot vector를 특정 크기의 `dim` 차원으로 embedding 시키는 layer.
*   `self.linear`: 변환된 embedding vector를 다시 원래 `vocab_size`로 바꾸는 layer.


In [15]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, dim):
    super(CBOW, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x):  # x: (B, 2W)
    embeddings = self.embedding(x)  # (B, 2W, d_w)
    embeddings = torch.sum(embeddings, dim=1)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

In [16]:
class SkipGram(nn.Module):
  def __init__(self, vocab_size, dim):
    super(SkipGram, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x): # x: (B)
    embeddings = self.embedding(x)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

두 가지 모델을 생성합니다.

In [17]:
cbow = CBOW(vocab_size=len(w2i), dim=256)
skipgram = SkipGram(vocab_size=len(w2i), dim=256)

### **모델 학습**

다음과 같이 hyperparamter를 세팅하고 `DataLoader` 객체를 만듭니다.

In [20]:
batch_size=4
learning_rate = 5e-4
num_epochs = 5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

cbow_loader = DataLoader(cbow_set, batch_size=batch_size)
skipgram_loader = DataLoader(skipgram_set, batch_size=batch_size)

첫번째로 CBOW 모델 학습입니다.

In [21]:
cbow.train()
cbow = cbow.to(device)
optim = torch.optim.SGD(cbow.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(cbow_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = cbow(x)  # (B, V)
 
    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  6%|▋         | 1/16 [00:00<00:02,  5.61it/s]

##################################################
Epoch: 1
Train loss: 5.0871124267578125
Train loss: 4.495680809020996
Train loss: 4.404603004455566
Train loss: 5.016420364379883
Train loss: 4.321847438812256
Train loss: 4.89340877532959
Train loss: 4.328012466430664
Train loss: 5.597177505493164


100%|██████████| 16/16 [00:00<00:00, 478.75it/s]

Train loss: 3.588991641998291
Train loss: 3.9179019927978516
Train loss: 4.84475564956665
Train loss: 4.809310436248779
Train loss: 3.8026514053344727
Train loss: 5.6193389892578125
Train loss: 3.7394587993621826
Train loss: 3.8403148651123047
##################################################
Epoch: 2
Train loss: 4.912628173828125
Train loss: 4.365656852722168
Train loss: 4.283516883850098
Train loss: 4.8920087814331055
Train loss: 4.200763702392578
Train loss: 4.617302894592285
Train loss: 4.142653942108154
Train loss: 5.453695297241211
Train loss: 3.476940393447876
Train loss: 3.7472763061523438
Train loss: 4.685545921325684
Train loss: 4.41632080078125
Train loss: 3.665771245956421
Train loss: 5.48996639251709
Train loss: 3.5916147232055664
Train loss: 3.7136001586914062
##################################################
Epoch: 3
Train loss: 4.7411346435546875
Train loss: 4.23769998550415
Train loss: 4.163978099822998
Train loss: 4.769026756286621
Train loss: 4.080937385559082
Trai

다음으로 Skip-gram 모델 학습입니다.

In [22]:
skipgram.train()
skipgram = skipgram.to(device)
optim = torch.optim.SGD(skipgram.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(skipgram_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = skipgram(x)  # (B, V)

    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/64 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 3.879277229309082
Train loss: 4.311881065368652
Train loss: 4.338291168212891
Train loss: 4.409280776977539
Train loss: 3.8724794387817383
Train loss: 3.9257640838623047
Train loss: 4.668275356292725
Train loss: 4.58461856842041
Train loss: 4.281807899475098
Train loss: 4.214297771453857
Train loss: 4.253112316131592
Train loss: 4.216219425201416
Train loss: 4.251491546630859
Train loss: 3.7652699947357178
Train loss: 3.8199732303619385
Train loss: 4.509627342224121
Train loss: 4.243988990783691
Train loss: 4.0105156898498535
Train loss: 3.913928508758545
Train loss: 3.8982949256896973
Train loss: 4.303450107574463
Train loss: 4.4208478927612305
Train loss: 4.666552543640137
Train loss: 4.006771087646484
Train loss: 3.597928762435913
Train loss: 4.507756233215332
Train loss: 4.740218162536621
Train loss: 4.451238632202148
Train loss: 3.972871780395508
Train loss: 4.088748455047607
Train loss: 4.291412353515625
Trai

 70%|███████   | 45/64 [00:00<00:00, 448.80it/s]

Train loss: 4.688884735107422
Train loss: 4.418670177459717
Train loss: 3.931244134902954
Train loss: 4.063446998596191
Train loss: 4.264033317565918
Train loss: 4.071428298950195
Train loss: 4.458360195159912
Train loss: 3.672696590423584
Train loss: 3.9069161415100098
Train loss: 4.234782695770264
Train loss: 4.500520706176758
Train loss: 3.8653693199157715
Train loss: 4.057750225067139
Train loss: 4.307658672332764
Train loss: 4.42566442489624
Train loss: 4.121435165405273
Train loss: 4.092445373535156
Train loss: 4.4557085037231445
Train loss: 3.958235263824463
Train loss: 4.194893836975098
Train loss: 4.084729194641113
Train loss: 4.297762870788574
Train loss: 4.272451877593994
Train loss: 3.9394729137420654
Train loss: 4.505410194396973
Train loss: 4.002418518066406
Train loss: 4.2336344718933105
Train loss: 4.626457214355469
Train loss: 4.467716217041016
Train loss: 3.943756580352783
Train loss: 4.034486770629883
Train loss: 4.134688377380371
Train loss: 4.0453386306762695
Train

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 3.9123969078063965
Train loss: 4.471397876739502
Train loss: 3.9579200744628906
Train loss: 4.202617645263672
Train loss: 4.593818664550781
Train loss: 4.436117172241211
Train loss: 3.9150567054748535
Train loss: 3.9760942459106445
Train loss: 4.105401515960693
Train loss: 4.017023086547852
Train loss: 3.829515218734741
Train loss: 4.060315132141113
Train loss: 4.068869590759277
Train loss: 4.519894599914551
Train loss: 3.5279757976531982
##################################################
Epoch: 4
Train loss: 3.8029375076293945
Train loss: 4.165946006774902
Train loss: 4.248966217041016
Train loss: 4.223013877868652
Train loss: 3.8019351959228516
Train loss: 3.834437131881714
Train loss: 4.5397491455078125
Train loss: 4.468124866485596
Train loss: 4.175765037536621
Train loss: 4.132532119750977
Train loss: 4.187439441680908
Train loss: 4.113443374633789
Train loss: 4.183045387268066
Train loss: 3.679044485092163
Train loss: 3.7344894409179688
Train loss: 4.427087783813477
T

100%|██████████| 64/64 [00:00<00:00, 431.94it/s]

Train loss: 4.498126029968262
Train loss: 4.4296875
Train loss: 4.140804290771484
Train loss: 4.105533123016357
Train loss: 4.1657304763793945
Train loss: 4.079675674438477
Train loss: 4.160921096801758
Train loss: 3.6506989002227783
Train loss: 3.7064223289489746
Train loss: 4.400124549865723
Train loss: 4.1359639167785645
Train loss: 3.8993258476257324
Train loss: 3.803755044937134
Train loss: 3.780463695526123
Train loss: 3.8628249168395996
Train loss: 4.068326950073242
Train loss: 4.42433500289917
Train loss: 3.8937864303588867
Train loss: 3.471031665802002
Train loss: 4.2561187744140625
Train loss: 4.537114143371582
Train loss: 4.324939727783203
Train loss: 3.8094940185546875
Train loss: 3.989232063293457
Train loss: 4.182858467102051
Train loss: 3.996727466583252
Train loss: 4.386472702026367
Train loss: 3.6137571334838867
Train loss: 3.814541816711426
Train loss: 4.130185604095459
Train loss: 4.305899143218994
Train loss: 3.73909854888916
Train loss: 3.953801393508911
Train loss

### **테스트**

학습된 각 모델을 이용하여 test 단어들의 word embedding을 확인합니다.

In [23]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = cbow.embedding(input_id)

  print(f"Word: {word}")
  print(emb.squeeze(0))

Word: 음식
tensor([-7.9042e-01, -7.0538e-01, -6.6976e-01,  8.6610e-01, -8.6829e-01,
         1.2286e+00,  4.8231e-01,  8.1646e-01, -1.9411e+00,  8.9980e-01,
        -1.7746e+00, -4.5992e-01, -1.3263e+00,  3.5081e-01,  1.5046e+00,
         8.1896e-01,  1.1918e-04, -5.0712e-01,  9.6802e-01,  2.5530e-01,
         8.3008e-01, -6.7137e-01, -8.4117e-01, -3.9905e-01, -8.0826e-01,
        -1.0100e+00, -1.4331e+00, -9.6992e-01, -2.0723e-01, -6.3269e-01,
        -8.3237e-01, -1.0523e+00,  9.5183e-01, -1.2437e+00,  2.6162e-01,
        -7.7515e-02,  9.3255e-01, -2.2127e-01, -4.1224e-01,  7.8853e-01,
         3.7518e-01, -2.2188e-01,  5.6199e-01,  2.7056e+00, -2.4016e-01,
         2.4015e-01,  1.5320e+00, -9.3272e-01,  4.3075e-01,  5.2494e-01,
        -2.5973e-01,  1.6622e+00,  1.6973e+00,  1.6079e-01, -6.5152e-02,
        -9.4314e-01, -6.1241e-01, -1.5328e-02, -3.0202e+00,  2.5711e-01,
        -3.6682e-01, -8.8679e-01, -3.9826e-01, -2.1968e-01,  4.7693e-02,
         7.9119e-01, -3.1150e-02,  5.4193e

In [24]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = skipgram.embedding(input_id)

  print(f"Word: {word}")
  print(max(emb.squeeze(0)))

Word: 음식
tensor(3.6042, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 맛
tensor(2.5437, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 서비스
tensor(3.2040, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 위생
tensor(2.8895, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 가격
tensor(2.2909, device='cuda:0', grad_fn=<UnbindBackward>)


In [ ]:
!apt-get install -qq texlive texlive-xetex texlive-latex-extra pandoc
!pip install -qq pypandoc

from google.colab import drive
drive.mount('/content/drive')

!jupyter nbconvert --to PDF '/content/drive/My Drive/Colab Notebooks/1_naive_bayes.ipynb의 사본'